In [1]:
import random

import matplotlib.pyplot as plt
from albumentations import (
    Compose, RandomBrightness, JpegCompression, HueSaturationValue, RandomContrast, HorizontalFlip,
    Rotate, VerticalFlip, Crop, PixelDropout, CropAndPad, RandomBrightnessContrast
)
import config
import paths
import randomness
from cnn_builder import *
from config import *
from dataset_functions import *
from functools import partial
from hypercube_set import HypercubeSet
from hypercube_loader import *
import numpy as np
import random
import randomness
import rendering
import training_history

import os

os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'
os.chdir(os.getcwd().split("jupyter")[0])

inf = 2e32

%load_ext autoreload
%autoreload 2
%pylab inline

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


C:\Users\allopezr\Documents\GitHub\VineyardUAVClassification\venv\lib\site-packages\IPython\core\magics\pylab.py:162: UserWarning:

pylab import has clobbered these variables: ['random', 'shuffle', 'inf', 'copy']
`%matplotlib` prevents importing * from pylab and numpy



2 Physical GPUs, 2 Logical GPUs


In [2]:
font_mapping = {'family': 'Palatino Linotype', 'weight': 'normal', 'size': 11}
plt.rc('font', **font_mapping)

In [3]:
hc_array_red, max_class_idx, _ = load_hypercubes(plot_hc=False, plot_mask=False, n_max_cubes=inf, folder='media/Mateus_2022/red/')
hc_array_white, _, color_dict = load_hypercubes(plot_hc=False, plot_mask=False, n_max_cubes=inf, folder='media/Mateus_2022/white/', baseline_class_idx=max_class_idx)

Reading media/Mateus_2022/red\raw_112_rf.hdr ...
Hypercube shape: (1872, 640, 270)
{(0, 0, 0): 0, (255, 128, 255): 1, (192, 128, 255): 2, (255, 128, 0): 3, (255, 255, 128): 4, (128, 255, 0): 5, (128, 255, 128): 6, (128, 255, 255): 7, (128, 128, 255): 8}
Reading media/Mateus_2022/red\raw_1984_rf.hdr ...
Hypercube shape: (1920, 640, 270)
{(0, 0, 0): 0, (255, 128, 255): 1, (192, 128, 255): 2, (255, 128, 0): 3, (255, 255, 128): 4, (128, 255, 0): 5, (128, 255, 128): 6, (128, 255, 255): 7, (128, 128, 255): 8}
Reading media/Mateus_2022/red\raw_3904_rf.hdr ...
Hypercube shape: (1840, 640, 270)
{(0, 0, 0): 0, (255, 128, 255): 1, (192, 128, 255): 2, (255, 128, 0): 3, (255, 255, 128): 4, (128, 255, 0): 5, (128, 255, 128): 6, (128, 255, 255): 7, (128, 128, 255): 8}
Reading media/Mateus_2022/red\raw_5744_rf.hdr ...
Hypercube shape: (1840, 640, 270)
{(0, 0, 0): 0, (255, 128, 255): 1, (192, 128, 255): 2, (255, 128, 0): 3, (255, 255, 128): 4, (128, 255, 0): 5, (128, 255, 128): 6, (128, 255, 255): 7, (

In [4]:
hc_set = HypercubeSet(hc_array_red + hc_array_white)
hc_set.print_num_samples()
hc_set.identify_ground_samples()
hc_set.split_hypercubes(test_percentage=.0)
transformation, standard_scaler = hc_set.standardize(num_features=config.num_target_features, selection_method=LayerSelectionMethod.FACTOR_ANALYSIS)

Number of samples per class:
Class 0: 12388109
Class 1: 144315
Class 2: 75078
Class 3: 53620
Class 4: 71547
Class 5: 58680
Class 6: 36114
Class 7: 35656
Class 8: 67157
Class 9: 242412
Class 10: 98304
Class 11: 77229
Class 12: 13556
Class 13: 105384
Class 14: 92432
Class 15: 101885
Class 16: 261228
Class 17: 44654
Train size: 145814, Test size: 0
Train size: 154362, Test size: 0
Train size: 119666, Test size: 0
Train size: 110729, Test size: 0
Train size: 11596, Test size: 0
Train size: 69155, Test size: 0
Train size: 168575, Test size: 0
Train size: 81132, Test size: 0
Train size: 226815, Test size: 0
Train size: 89462, Test size: 0
Train size: 227458, Test size: 0
Train size: 63593, Test size: 0
Train size: 110894, Test size: 0
|████████████████████████████████████████| 52/52 [100%] in 1:09.4 (0.75/s)                                              ▅▇ 23/52 [44%] in 17s (~21s, 1.4/s) 


In [5]:
hc_array_white_2021, _, _ = load_hypercubes(plot_hc=False, plot_mask=False, n_max_cubes=inf, folder='media/Mateus_2021/white/', color_dict=color_dict, baseline_class_idx=max_class_idx)

Reading media/Mateus_2021/white\raw_10208_rf.hdr ...
Hypercube shape: (2000, 640, 270)
{(0, 0, 0): 0, (128, 255, 128): 1, (128, 255, 255): 2, (128, 128, 255): 3, (128, 255, 0): 4, (255, 255, 128): 5, (255, 128, 0): 6, (192, 128, 255): 7, (255, 128, 255): 8, (0, 0, 255): 9}
Reading media/Mateus_2021/white\raw_12400_rf.hdr ...
Hypercube shape: (2000, 640, 270)
{(0, 0, 0): 0, (128, 255, 128): 1, (128, 255, 255): 2, (128, 128, 255): 3, (128, 255, 0): 4, (255, 255, 128): 5, (255, 128, 0): 6, (192, 128, 255): 7, (255, 128, 255): 8, (0, 0, 255): 9}
Reading media/Mateus_2021/white\raw_14400_rf.hdr ...
Hypercube shape: (2000, 640, 270)
{(0, 0, 0): 0, (128, 255, 128): 1, (128, 255, 255): 2, (128, 128, 255): 3, (128, 255, 0): 4, (255, 255, 128): 5, (255, 128, 0): 6, (192, 128, 255): 7, (255, 128, 255): 8, (0, 0, 255): 9}
Reading media/Mateus_2021/white\raw_16592_rf.hdr ...
Hypercube shape: (2000, 640, 270)
{(0, 0, 0): 0, (128, 255, 128): 1, (128, 255, 255): 2, (128, 128, 255): 3, (128, 255, 0): 4

In [6]:
hc_set = HypercubeSet(hc_array_white_2021)
hc_set.print_num_samples()

Number of samples per class:
Class 0: 10953299
Class 1: 0
Class 2: 0
Class 3: 0
Class 4: 0
Class 5: 0
Class 6: 0
Class 7: 0
Class 8: 0
Class 9: 412297
Class 10: 136221
Class 11: 111267
Class 12: 139110
Class 13: 166665
Class 14: 133336
Class 15: 91418
Class 16: 223278
Class 17: 433109


In [7]:
hc_set.identify_ground_samples()
hc_set.split_hypercubes(test_percentage=.0)
hc_set.standardize(num_features=config.num_target_features, selection_method=LayerSelectionMethod.FACTOR_ANALYSIS, standard_scaler=standard_scaler, transformation=transformation)

Train size: 264828, Test size: 0
Train size: 245153, Test size: 0
Train size: 245510, Test size: 0
Train size: 248290, Test size: 0
Train size: 246433, Test size: 0
Train size: 123439, Test size: 0
Train size: 174447, Test size: 0
Train size: 78047, Test size: 0
Train size: 53095, Test size: 0
Train size: 167459, Test size: 0
|████████████████████████████████████████| 40/40 [100%] in 43.5s (0.92/s)                                                ▄▆█ 32/40 [80%] in 24s (~6s, 1.3/s)  ▃▁▃ 33/40 [82%] in 26s (~6s, 1.3/s) 


(FactorAnalysis(n_components=40, random_state=42), StandardScaler())

In [8]:
network_type = 'allopezr_2d'
read_json_config(paths.config_file, network_type=network_type)
network_name = get_name(network_type)

In [9]:
import papers

# Read network weights
network_weights_file = os.path.join(paths.result_folder + 'network/', network_name + "*.h5")
network_weights_files = glob.glob(network_weights_file)
if len(network_weights_files) > 0:
    network_weights_file = network_weights_files[-1]
    print("Loading weights file " + network_weights_file)
    model = keras.models.load_model(network_weights_file,
                                    custom_objects={'SpatialAttention': papers.aspn.SpatialAttention,
                                                    'SecondOrderPooling': papers.aspn.SecondOrderPooling})

    model.summary()

Loading weights file results/uav/network\allopezr_2d_23x22_16_window_size_test_0.h5
Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 23, 23, 40)  0           []                               
                                ]                                                                 
                                                                                                  
 reshape_18 (Reshape)           (None, 529, 40)      0           ['input_10[0][0]']               
                                                                                                  
 lambda_9 (Lambda)              (None, 529, 40)      0           ['reshape_18[0][0]']             
                                                                                                  
 spatial

In [10]:
num_classes = hc_set.get_num_classes() - 1
img_shape = (config.patch_size, config.patch_size, config.num_target_features)
num_iterations = config.num_training_splits * 20
percentage_step = 1.0 / num_iterations

In [11]:
import gc
import training_metrics

#### Split test into batches
y_test_global = []
test_prediction_global = []
test_prediction_prob_global = []

for batch in range(config.num_test_splits):
    X_test, y_test = hc_set.split(patch_size=config.patch_size, patch_overlap=config.patch_overlapping,
                                  train=True, start_percentage=percentage_step * batch,
                                  end_percentage=percentage_step * (batch + 1))

    test_prediction_prob = model.predict(X_test)
    test_prediction = np.argmax(test_prediction_prob, axis=1)

    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    print("Test Loss: " + str(test_loss) + ", Test Accuracy: " + str(test_accuracy))

    # y test to int
    y_test_global.extend(np.asarray(y_test, dtype="int64").tolist())
    test_prediction_global.extend(test_prediction.tolist())
    test_prediction_prob_global.extend(test_prediction_prob.tolist())

    del X_test, y_test, test_prediction, test_prediction_prob
    gc.collect()

y_test_global = np.asarray(y_test_global)
test_prediction_global = np.asarray(test_prediction_global)
test_prediction_prob_global = np.asarray(test_prediction_prob_global)

metrics = training_metrics.TrainingMetrics()
metrics.append_evaluation(y_test_global, test_prediction_global, test_prediction_prob_global)
metrics.print_metrics()

# Graphic results
render_confusion_matrix(y_test_global, test_prediction_global, model_name=network_name)

305/305 [==============================] - 3s 9ms/step - loss: 155.3234 - sparse_categorical_accuracy: 0.0700
Test Loss: 155.32342529296875, Test Accuracy: 0.07002973556518555
306/306 [==============================] - 3s 9ms/step - loss: 156.5564 - sparse_categorical_accuracy: 0.0625
Test Loss: 156.55642700195312, Test Accuracy: 0.062480825930833817
306/306 [==============================] - 3s 9ms/step - loss: 154.6658 - sparse_categorical_accuracy: 0.0682
Test Loss: 154.665771484375, Test Accuracy: 0.06816785782575607
305/305 [==============================] - 3s 9ms/step - loss: 156.2232 - sparse_categorical_accuracy: 0.0680
Test Loss: 156.2231903076172, Test Accuracy: 0.06798605620861053
306/306 [==============================] - 3s 9ms/step - loss: 155.4495 - sparse_categorical_accuracy: 0.0626
Test Loss: 155.44949340820312, Test Accuracy: 0.06262151151895523


ValueError: Number of classes in y_true not equal to the number of columns in 'y_score'